# Feedforward Neural Networks — Training and Evaluating the Model on Forex Data
a medium.com article

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
# from tensorflow.keras.models import Sequential
from keras.src.models import Sequential
# from tensorflow.keras.layers import Dense
from keras.src.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
# Load the forex data
url = "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=off&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=DTWEXB&scale=left&cosd=1999-01-04&coed=2021-09-10&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Daily&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2021-09-16&revision_date=2021-09-16&nd=1971-01-04"
data = pd.read_csv(url, index_col='DATE', parse_dates=True)
data = data.dropna()

In [ ]:
# Remove rows with '.' and convert the column to float
data = data[data.DTWEXB != '.']
data['DTWEXB'] = data['DTWEXB'].astype(float)

# Scale the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Create lagged feature
X = data_scaled[:-1]
y = data_scaled[1:]

# Split the data into training and test sets
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
# Create the model
model = Sequential([
    Dense(16, activation='relu', input_shape=(1,)),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)

In [ ]:
# Make predictions on the test
y_pred = model.predict(X_test)

# Inverse transform the predictions and true values
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test_inv, y_pred_inv)
print(f'Mean Squared Error: {mse}')

In [ ]:
import matplotlib.pyplot as plt

# Plot the true and predicted exchange rates
plt.figure(figsize=(12, 6))
plt.plot(data.index[train_size+1:], y_test_inv, label='True Exchange Rate')
plt.plot(data.index[train_size+1:], y_pred_inv, label='Predicted Exchange Rate')
plt.xlabel('Date')
plt.ylabel('Exchange Rate')
plt.title('EUR/USD Exchange Rate: True vs. Predicted')
plt.legend()
plt.show()